# EKSTRAKSI KATA KUNCI

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import display
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
df = pd.read_csv('/content/drive/MyDrive/datamining/tugas/beritabanten (1).csv')
df

,Judul,Isi,Tanggal,Kategori
0,Sekretaris KONI Kota Serang Jadi Bacalon Tungg...,SERANG– Satu kandidat calon Ketua Umum (Ketum)...,"Rabu 9 Agu 2023, 19:02 WIB",Sport
1,Prediksi Irak vs Timnas Indonesia Kualifikasi ...,SERANG– Pertandingan Grup F Kualifikasi Piala ...,"Kamis 16 Nov 2023, 04:17 WIB",Sport
2,Dua Pengedar Obat Tanpa Izin Edar Ditangkap Po...,PANDEGLANG– Satuan Reserse Narkoba (Satresnark...,"Sabtu 16 Sep 2023, 11:46 WIB",hukum
3,FIFA Cabut Status Indonesia Sebagai Tuan Rumah...,JAKARTA– Indonesia batal menggelar Piala Dunia...,"Kamis 30 Mar 2023, 00:12 WIB",Sport
4,Tiga Pengurus Panahan Banten Dilirik Pusat,SERANG– Ketua Umum Pengurus Provinsi Persatuan...,"Minggu 12 Feb 2023, 12:08 WIB",Sport
...,...,...,...,...
1195,Sosialisasi E-Sport di Cilegon Merambah ke Ber...,CILEGON– Demam Electronic Sport atau E-Sport k...,"Senin 17 Apr 2023, 12:07 WIB",Sport
1196,Kontingen Atlet Basket U-15 Banten Bersaing di...,"KAB. TANGERANG– Ketua Umum Perbasi Banten, Ahm...","Jumat 6 Jan 2023, 05:07 WIB",Sport
1197,"Rencana Pertemuan SBY dan Megawati, Buka Pelua...",SERANG– Ketua Majelis Tinggi Partai Demokrat S...,"Selasa 5 Sep 2023, 03:10 WIB",Politik
1198,10 Kejutan Piala Dunia Terbesar Teratas,SERANG– Setelah kemenangan yang mengejutkan sa...,"Kamis 24 Nov 2022, 07:13 WIB",Sport


In [28]:
corpus = stopwords.words('indonesian')
def stopwords(text):
  return ' '.join(word for word in text.split() if word not in corpus)

In [29]:
def plot_graph(G, figsize=(35, 30), node_size=700, node_color='skyblue'):
  # Menggambar graf dengan canvas yang diperbesar
  pos = nx.spring_layout(G)  # Menentukan posisi simpul
  labels = nx.get_edge_attributes(G, 'weight')

  # Menentukan ukuran canvas
  plt.figure(figsize=figsize)

  # Menggambar graf dengan ukuran canvas yang diperbesar
  nx.draw(G, pos, with_labels=True, node_size=node_size, node_color=node_color)
  nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_color='red')
  plt.show()

In [30]:
def graph_co_occurrence(x, show_matrics=False):
  vectorizer = CountVectorizer()
  tfidf_matrics = vectorizer.fit_transform(x)
  co_occurrence_matrix = tfidf_matrics.T.dot(tfidf_matrics).toarray()
  df_co_occurence = pd.DataFrame(co_occurrence_matrix, columns=vectorizer.get_feature_names_out())
  df_co_occurence.insert(0, 'Word', vectorizer.get_feature_names_out())

  if show_matrics:
    print("Tabel Co-occurence Matrics: ")
    display(df_co_occurence)
    print()

  # Membuat graf berbobot menggunakan NetworkX
  G = nx.DiGraph()
  num_words = co_occurrence_matrix.shape[0]

  # Menambahkan edge ke graf berdasarkan matriks co-occurrence
  for i in range(num_words):
    for j in range(i + 1, num_words):
      weight = co_occurrence_matrix[i, j]
      if weight > 0:
        G.add_edge(vectorizer.get_feature_names_out()[i], vectorizer.get_feature_names_out()[j], weight=weight)
  return G

In [31]:
def text_rank(graph, damping_factor=0.85, max_iter=100, tol=1e-4):
  # Inisialisasi nilai TextRank untuk setiap node
  initial_value = 1 / graph.number_of_nodes()
  ranks = {node: initial_value for node in graph.nodes()}

  print(f"Inisialisasi Score Awal : {ranks}")
  print(f"Edges : {graph.edges()}")
  print()

  for i in range(max_iter):
    new_ranks = {}
    print(f"========== Iterasi Ke - {i} ==========")

    for node in graph.nodes():
      rank_sum = 0
      operasi = ''

      # Menampilkan informasi out_sum dan win untuk setiap node dan neighbor
      out_sums_info = {}
      win = graph.in_degree(node, weight="weight")

      for neighbor in graph.predecessors(node):
        edge_weight = graph[neighbor][node]["weight"]
        out_sum = sum(graph[neighbor][out_neighbor]["weight"] for out_neighbor in graph.successors(neighbor))
        rank_sum += (edge_weight / out_sum) * ranks[neighbor]
        operasi += f"({edge_weight}/{out_sum}) * {ranks[neighbor]} + "

        # Menambah informasi out_sum untuk setiap neighbor
        out_sums_info[neighbor] = {out_neighbor: graph[neighbor][out_neighbor]["weight"] for out_neighbor in graph.successors(neighbor)}

      # Jika operasi kosong, atur nilai operasi menjadi 0
      operasi = operasi[:-3] if operasi else '0'

      new_rank = (1 - damping_factor) + damping_factor * rank_sum
      new_ranks[node] = new_rank

      # Menampilkan informasi out_sum dan win pada setiap iterasi
      print(f"Win({node}): {win}")
      print(f"Out_sums_info({node}): {out_sums_info}")
      print(f"W({node}) = (1 - {damping_factor}) + {damping_factor} * ({operasi}) = {new_rank}")
      print()

    # Periksa konvergensi
    convergence = all(abs(new_ranks[node] - ranks[node]) < tol for node in graph.nodes())
    ranks = new_ranks
    print(f"New Score : {new_ranks}")
    print()

    if convergence:
      print(f"Konvergensi tercapai pada iterasi ke-{i}")
      break

  return ranks

In [32]:
def extract_top_words(x, w=3, show_matrics=False, show_scores=False, index=None):
    full_text = ' '.join(word for word in x)

    # Stopwords removal
    x = [stopwords(full_text)]

    G = graph_co_occurrence(x, show_matrics)

    # Menghitung nilai dari PageRank (TextRank)
    scores = text_rank(G)

    # Dictionary untuk menyimpan skor tertinggi setiap kata
    ranked_words_dict = {}

    for word in ' '.join(x).split():
        current_score = scores.get(word, 0)
        if word not in ranked_words_dict or current_score > ranked_words_dict[word]:
            ranked_words_dict[word] = current_score

    # Mengurutkan kata-kata berdasarkan skor tertinggi
    ranked_words = sorted(((score, word) for word, score in ranked_words_dict.items()), key=lambda x: (x[0], x[1]), reverse=True)

    # Memilih sejumlah w kata tertinggi
    selected_words = [word for _, word in ranked_words[:w]] if w is not None else None

    # Menggabungkan kata-kata menjadi satu string terpisah dengan koma
    keywords = ', '.join(selected_words) if selected_words else ''

    if show_scores:
      print(f'Dokumen ke {index} : {full_text}')
      print(f'{w} Kata Kunci : {keywords}')
      print("TextRank Scores:")
      for score, word in ranked_words:
          print(f"Skor: {score}, Kata: {word}")

    return (G, selected_words)

In [ ]:
data1=df[1:2]
data1

In [ ]:
text = data1['Isi']
G, keywords = extract_top_words(text, w=2, show_matrics=True)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
plot_graph(G, figsize=(5, 5), node_size=700,)

In [ ]:
keywords

In [ ]:
text1 = ['saya berolahraga pagi hari']
G1, keywords1 = extract_top_words(text1, w=4, show_matrics=True)

In [ ]:
plot_graph(G1, figsize=(5, 5), node_size=700,)

In [ ]:
keywords1